| 단계                                            | 목적                 | 이유                                 |
| --------------------------------------------- | ------------------ | ---------------------------------- |
| 1️⃣ **라이트 정제**<br>(Unicode NFKC, 공백·URL 제거 등) | 요약 입력을 깨끗하게        | KoBART가 HTML 잔여·줄바꿈·URL에 민감 → 품질 ↓ |
| 2️⃣ **1차 중복 제거**<br>(`link`·`title+datetime`) | 불필요한 인퍼런스 절감       | 요약·감성 분석 시간은 텍스트 건수에 비례            |
| 3️⃣ **KoBART 요약**                             | `summary` 컬럼 생성    | 이후 단계에서 짧은 텍스트 활용 → RAM·CPU 부담↓    |
| 4️⃣ **감성 분석**<br>(KB-ALBERT·FinBERT 등)        | `neg/neu/pos` 스코어  | 짧은 summary를 쓰면 분석 속도 2-3배 개선       |
| 5️⃣ **최종 CSV 저장**                             | `raw_text`까지 보존 권장 | 추후 디버깅·재학습 대비                      |


위 절차 중 현재 파일에서는 1,2,3번 수행

In [ ]:
# 코랩에서 실행

#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
news_clean_20250805.csv → KoBART 요약(긴 문서 슬라이딩 윈도우) → text 컬럼 제거
출력: news_summary_20250805.csv
"""

from __future__ import annotations
import math
from pathlib import Path

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm

# ───────────────────────────────
# 1) 경로·하이퍼파라미터
# ───────────────────────────────
# Colab 예시
IN_CSV  = "/content/drive/MyDrive/Invemotion/news_clean_20250805.csv"
OUT_CSV = "/content/drive/MyDrive/Invemotion/news_summary_20250805(2).csv"

# 로컬 맥OS 예시 (원하면 위/아래 중 하나만 사용)
# IN_CSV  = "/Users/yujimin/KB AI CHALLENGE/project/news_clean_20250805.csv"
# OUT_CSV = "/Users/yujimin/KB AI CHALLENGE/project/news_summary_20250805.csv"

BATCH_SIZE      = 8           # T4(16GB)에서 8~16 권장
MAX_INPUT_LEN   = 512
MAX_SUMMARY_LEN = 128
DEVICE          = "cuda" if torch.cuda.is_available() else "cpu"

CHUNK_TOKENS    = 400         # 긴 문서 슬라이딩 윈도우 조각 크기
OVERLAP_TOKENS  = 80          # 조각 간 겹침

# ───────────────────────────────
# 2) 모델 로드
# ───────────────────────────────
print("🔄  Loading KoBART summarizer …")
tok   = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization").to(DEVICE)
if DEVICE == "cuda":
    model = model.to(dtype=torch.float16)  # VRAM 절약
model.eval()

@torch.inference_mode()
def summarize_batch(texts: list[str]) -> list[str]:
    """KoBART 배치 요약 (token_type_ids 제거)"""
    enc = tok(
        texts,
        max_length=MAX_INPUT_LEN,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    enc.pop("token_type_ids", None)
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    outs = model.generate(
        **enc,
        max_length=MAX_SUMMARY_LEN,
        num_beams=4,
        early_stopping=True,
        # 품질 안정 옵션(원하면 해제/조정)
        no_repeat_ngram_size=3
    )
    return tok.batch_decode(outs, skip_special_tokens=True)

def chunk_by_tokens(text: str,
                    chunk_tokens=CHUNK_TOKENS,
                    overlap=OVERLAP_TOKENS) -> list[str]:
    """토큰 단위로 슬라이싱하여 부분 텍스트 리스트 반환"""
    ids = tok.encode(text, add_special_tokens=True)
    chunks = []
    step = max(1, chunk_tokens - overlap)
    for s in range(0, len(ids), step):
        piece = ids[s:s + chunk_tokens]
        if not piece:
            break
        chunks.append(tok.decode(piece, skip_special_tokens=True))
        if s + chunk_tokens >= len(ids):
            break
    return chunks

def summarize_long(text: str) -> str:
    """긴 문서: 조각별 요약 → 메타 요약(map-reduce)"""
    parts = chunk_by_tokens(text)
    if not parts:
        return ""
    # 1차: 조각 요약 (배치 처리)
    part_sums = []
    for i in range(0, len(parts), BATCH_SIZE):
        part_sums += summarize_batch(parts[i:i + BATCH_SIZE])
    # 2차: 요약들의 요약
    final = summarize_batch([" ".join(part_sums)])[0]
    return final

# ───────────────────────────────
# 3) 데이터 로드
# ───────────────────────────────
df = pd.read_csv(IN_CSV, encoding="utf-8")
assert "text" in df.columns, "입력 CSV에 'text' 컬럼이 필요합니다."
df["text"] = df["text"].fillna("")

print(f"📂 Loaded {len(df):,} rows")

# ───────────────────────────────
# 4) 512토큰 초과 여부 측정
# ───────────────────────────────
enc = tok(df["text"].tolist(), truncation=False, padding=False)
df["tok_len"]   = [len(x) for x in enc["input_ids"]]
df["truncated"] = df["tok_len"] > MAX_INPUT_LEN
print(f"⚙️  >512 tokens: {df['truncated'].mean():.1%}")

# ───────────────────────────────
# 5) 요약 실행
#  - 짧은 문서(≤512): 배치 요약
#  - 긴 문서(>512):  슬라이딩 윈도우 요약
# ───────────────────────────────
summaries = [""] * len(df)

# 5-a) 짧은 문서 일괄 처리
short_idx = df.index[~df["truncated"]].tolist()
for i in tqdm(range(0, len(short_idx), BATCH_SIZE), desc="Short docs"):
    batch_ids = short_idx[i:i + BATCH_SIZE]
    texts = df.loc[batch_ids, "text"].tolist()
    outs = summarize_batch(texts)
    for j, idx in enumerate(batch_ids):
        summaries[idx] = outs[j]

# 5-b) 긴 문서 개별 처리 (속도보다 품질 우선)
long_idx = df.index[df["truncated"]].tolist()
for idx in tqdm(long_idx, desc="Long docs"):
    summaries[idx] = summarize_long(df.at[idx, "text"])

# ───────────────────────────────
# 6) 저장
# ───────────────────────────────
df.insert(2, "summary", summaries)
df = df.drop(columns=["text", "tok_len", "truncated"])

out_path = Path(OUT_CSV)
out_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"🎉 Saved → {out_path}  ({len(df):,} rows, cols: {list(df.columns)})")